<h3 align="center"><b>How To Calculate Memory Usage Per Process On Linux?</h3>

> **Pss is Best Memory Indiactor Memory useage per process.**

> **Concept of PSS Memory :**<br>
**RSS = Private pages + Shared pages**<br>
**PSS = Private pages + Shared pages / Number process shared**<br>
**Actual RAM usage = Private pages + PSS - Private pages**<br>


In [3]:
%%bash

PID=2489 ##This is Processid
awk '/^Pss:/{ sum += $2 } END {printf "%.1f MB\n", sum/1024}' \
/proc/$PID/smaps

42.7 MB


In [4]:

from pathlib import Path
PID='2489'
file = Path('/proc/'+PID+'/smaps').read_text()
Pss = [int(i.strip().split()[1]) for i in file.splitlines()
if i.startswith("Pss")]
print(round(sum(Pss)/1024, 1), 'MB')

42.7 MB


<h4 align="center"><b>Calculate Private, Shared Memory Per Process</h4>

In [5]:
%%bash

PID=2489

Pss=$(awk '/^Pss:/{ sum += $2 } END {print sum/1024}' /proc/$PID/smaps)
Private=$(awk '/^Private/{ sum += $2 } END {print sum/1024}' /proc/$PID/smaps)

if [ x"$Rss" != "x" -o x"$Private" != "x" ]; then
Shared=$(awk -v pss=$Pss -v pri=$Private 'BEGIN {print pss - pri}')
Name=$(awk '/^Name/{print $2}' /proc/$PID/status)
Sum=$(awk -v sha=$Shared -v pri=$Private 'BEGIN {print sha + pri}')
echo -e "Private(MB)      Shared(MB)    = Total(MB) \t Process"
printf "%0.s=" {1..57};echo
echo -e "$Private   +      $Shared         = $Sum  \t $Name"
fi

Private(MB)      Shared(MB)    = Total(MB) 	 Process
39.0664   +      3.6748         = 42.7412  	 nautilus


In [16]:
#%%python3

from pathlib import Path
import sys

pss = []
private = []

try:
    pid = input('Enter Process id: ')    
    osmaps = Path('/proc/'+pid+'/smaps').read_text()  
except (ValueError, FileNotFoundError):
    print(sys.exc_info()[1])
    sys.exit(2)
else:
    for text in osmaps.splitlines():
        if text.startswith('Pss'):
            pss.append(int(text.split()[1]))
        if text.startswith('Private'):
            private.append(int(text.split()[1]))

Pss = round(sum(pss)/1024)
Private = round(sum(private)/1024)
Shared = Pss - Private
Total = Private + Shared
pname = Path('/proc/'+pid+'/status').read_text().splitlines()[0].split()[1]
print(f'{"Private(MB)"} {"Shared(MB)"} = {"Total(MB)"}  {"Process"}')
print('='*45)
print(f'{Private:^10} + {Shared:^10} {Total:^10} {pname:^10}')

Enter Process id: 2489
Private(MB) Shared(MB) = Total(MB)  Process
    39     +     4          43      nautilus 
